In [1]:
!pip install langchain langchain_commnunity langchain_text_splitters bs4

ERROR: Could not find a version that satisfies the requirement langchain_commnunity (from versions: none)

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for langchain_commnunity


In [30]:
import bs4
import os
from langchain.agents import AgentState, create_agent
from langchain_community.document_loaders import WebBaseLoader
from langchain.messages import MessageLikeRepresentation
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.tools import tool

In [31]:
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")
        )
    ),
)

docs = loader.load()

In [32]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [33]:
os.environ['OPENAI_API_KEY'] = '<paste-your-open-ai-key>'

In [34]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [35]:
vector_store = InMemoryVectorStore(embeddings)

In [36]:
_ = vector_store.add_documents(documents=all_splits)

In [37]:
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

In [38]:
@tool(response_format = "content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs =  vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
    (f"Source: {doc.metadata}\nContent: {doc.page_content}")
    for doc in retrieved_docs
    )
    return serialized, retrieved_docs

tools = [retrieve_context]

prompt=(
    "You have access to a tool that retrieve context from a blog post. "
    "Use the tool to help answer user queries."
)

agent = create_agent(model,tools, system_prompt=prompt)

In [39]:
query = "What is the task decomposition?"

for step in agent.stream(
    {"messages" : [{"role": "user", "content" : query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the task decomposition?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_0RtHlArJs8SP3uiKoHyS9tnM)
 Call ID: call_0RtHlArJs8SP3uiKoHyS9tnM
  Args:
    query: task decomposition
================================= Tool Message =================================
Name: retrieve_context

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}
Content: Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
Another quite distinct approach, LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning. This approach utilizes the Planning Domain Definition Language (PDDL) as an intermedi